Мы заходим в позицию по активу в момент выхода новости. Рассмотрим сценарий, в котором мы будем заходить в позицию только по новостям с позитивным таргетом.

In [1]:
import numpy as np
import pandas as pd
import os
import h5py

Из targets.hd5 извлечем нужные нам новости

In [2]:
targets = pd.read_hdf('targets-3.hd5', key='df')
targets['10_True_0.05'].value_counts()


0.0    36006
1.0      540
Name: 10_True_0.05, dtype: int64

Итого имеем 540 интересующих нас новостей, выделим их

In [3]:
targets = targets[targets['10_True_0.05'] == 1].drop(columns = ['10_True_0.05'])

In [4]:
targets

,news_id,ticker
45,0bbb3403279779b81fe6bc1fe556ff69,CTSO
117,1b754b1268deb3b6b6ed6a83ee7104ac,HALL
134,1ef7ccd09c652ee1bcc653eb9a9d857c,CAPR
191,2b3788b24dc64e622ec4b47b19de2521,SLGG
237,33b7d9486f20c0a6c296c8860c6bf00f,ABNB
...,...,...
35918,f27639fb94b4d032620d815e47d610d2,LW
36184,595e96d8289fd6df85dd64716680872f,BCYC
36322,95a136d40f47c8db3c0a851991f38fc7,REDU
36434,d573a87ede50634009cbb32ba331dca9,AGRX


Из файла ticker_price_moment_exit.hd5 извлечем цены на тикеры в момент выхода наших новостей

In [5]:
ticker_prices = pd.read_hdf('ticker_price_moment_exit.hd5', key='df')
ticker_prices.head()

,ticker,bidPrice,bidSize,askPrice,askSize,lastUpdated,lastSalePrice,lastSaleSize,lastSaleTime,volume,current_price,timestamp,news_id
0,VS,0.0,0.0,0.0,0.0,1629316800000.0,4.33,6.0,1629316790023,46.0,4.24,1629716406,0000d467d1a52a6a84fc8b30adbe8ec2
1,VSSYW,0.0,0.0,0.0,0.0,1628193600000.0,1.55,25.0,1628178157384,25.0,2.08,1629716406,0000d467d1a52a6a84fc8b30adbe8ec2
3,BTBT,0.0,0.0,0.0,0.0,1632513600000.0,9.44,1.0,1632513582057,38333.0,9.46,1632747622,0007ea6c22fc24e1f34021185ad07e4e
4,CAN,0.0,0.0,0.0,0.0,1632515322537,6.16,80.0,1632514484013,453506.0,6.11,1632747622,0007ea6c22fc24e1f34021185ad07e4e
5,HVBT,0.0,0.0,0.0,0.0,1631563200000.0,3.06,300.0,1631563186154,33347.0,3.06,1632747622,0007ea6c22fc24e1f34021185ad07e4e


In [6]:
selected_tickers = ticker_prices[ticker_prices['news_id'].isin(targets['news_id'])]
selected_tickers[['ticker', 'current_price', 'news_id']].head()

,ticker,current_price,news_id
650,FLGC,8.68,02bef0eac6ffefad5ed09dbbe10285a3
695,FTRP,4.81,02e52550c5a219eab849a8dd4ceabf1e
751,MMSI,71.65,032371cc3e6282c2a4b879dc530bfdf5
1096,CEI,1.725,04a3c3a7669c55c3f3139e66785ec07d
1097,GSAT,2.115,04a3c3a7669c55c3f3139e66785ec07d


In [7]:
selected_tickers.shape

(549, 13)

Теперь из файлов в папке intraday_data достанем цены закрытия соответсвующих тикеров в дни выхода новостей и узнаем, какие из них принесли нам прибыль

In [8]:
## Раньше работал этот способ, теперь не работает

In [9]:
files = [f for f in os.listdir('intraday_data/')]

In [10]:
all_data = []
for file in files:
    days_data = pd.read_hdf(f'intraday_data/{file}', key='df')
    all_data.append(days_data_df)

all_data['date'] = pd.to_datetime(all_data['time'], unit='s').df.date

UnknownTimeZoneError: 'cdateutil.tz.win\ntzwin\np0\n(VUTC\np1\ntp2\nRp3\n.'

Другой способ

In [11]:
files = [f for f in os.listdir('intraday_data/')]

In [14]:
days_data = h5py.File(f'intraday_data/{files[0]}','r+')
d_data = days_data['df']
all_data = pd.DataFrame([[0, 0, 0, 0, 0]], columns=np.array(d_data['block0_items']))

for file in files:
    days_data = h5py.File(f'intraday_data/{file}','r+')
    d_data = days_data['df']
    df0 = pd.DataFrame(np.array(d_data['block0_values']),
                       columns=np.array(d_data['block0_items'])) #здесь лежат ценовые данные по тинкеру
    df1 = pd.DataFrame(np.array(d_data['block1_values']),
                       columns=np.array(d_data['block1_items'])) #здесь лежат времена наблюдения
    day_data = pd.concat([df0, df1], axis=1)
    all_data = pd.concat([all_data, day_data], ignore_index=True)

In [15]:
all_data.shape

(19497011, 6)

In [16]:
all_data.tail(5)

,b'open',b'high',b'close',b'low',b'volume',b'time'
19497006,11.93,11.93,11.93,11.93,940.0,1.631318e+09
19497007,11.94,11.94,11.93,11.93,4285.0,1.631318e+09
19497008,11.93,11.95,11.95,11.93,7081.0,1.631318e+09
19497009,11.95,11.95,11.93,11.92,5663.0,1.631318e+09
19497010,11.92,11.93,11.89,11.89,11113.0,1.631318e+09


Теперь, по идее, необходимо по соответствию тинкеров и дат публикации новости извлечь из полученного нами датасета all_data цену закрытия для каждого наименования из датасета selected_tickers. Однако извлечь тинкеры из interaday_data данных не удалось и вот почему:

Проблема c тинкерами на примере одного файла

In [17]:
filename = 'intraday_data/2021-10-01.hd5'

h5 = h5py.File(filename,'r+')

data = h5['df']
h5['df'].keys()

<KeysViewHDF5 ['axis0', 'axis1', 'block0_items', 'block0_values', 'block1_items', 'block1_values', 'block2_items', 'block2_values', 'block3_items', 'block3_values', 'block4_items', 'block4_values']>

In [18]:
df0 = pd.DataFrame(np.array(data['block0_values']), columns=np.array(data['block0_items'])) #здесь лежат ценовые данные по тинкеру
df1 = pd.DataFrame(np.array(data['block1_values']), columns=np.array(data['block1_items'])) #здесь лежат времена наблюдения

In [19]:
col = pd.DataFrame([[0, 0, 0, 0, 0]], columns=np.array(data['block0_items']))
col

,b'open',b'high',b'close',b'low',b'volume'
0,0,0,0,0,0


In [20]:
data['block0_values'].shape

(471604, 5)

In [21]:
np.array(data['block2_values'])
#тут тинкеры, и в них происходит что-то странное, не названия, а цифры.
#Не очень понятно как с таким работать, поэтому я их не включил
#Однако найти соответсвие между данными без названия тинкера невозможно

array([array([128,   4, 149, ..., 148,  98,  46], dtype=uint8)],
      dtype=object)

In [22]:
all_data = pd.concat([df0, df1], axis=1)

In [23]:
pd.concat([all_data, col], ignore_index=True)

,b'open',b'high',b'close',b'low',b'volume',b'time'
0,13.12,13.12,12.73,12.73,5799.0,1.633087e+09
1,12.73,NaN,12.73,NaN,NaN,1.633087e+09
2,12.73,NaN,12.73,NaN,NaN,1.633087e+09
3,12.73,NaN,12.73,NaN,NaN,1.633087e+09
4,12.73,NaN,12.73,NaN,NaN,1.633087e+09
...,...,...,...,...,...,...
471600,143.32,143.32,143.32,143.32,100.0,1.633126e+09
471601,143.32,NaN,143.32,NaN,NaN,1.633126e+09
471602,143.32,NaN,143.32,NaN,NaN,1.633126e+09
471603,143.55,143.55,143.55,143.55,100.0,1.633126e+09
